# Projeto Python IA: Inteligência Artificial e Previsões

### Case: Score de Crédito dos Clientes

Você foi contratado por um banco para conseguir definir o score de crédito dos clientes. Você precisa analisar todos os clientes do banco e, com base nessa análise, criar um modelo que consiga ler as informações do cliente e dizer automaticamente o score de crédito dele: Ruim, Ok, Bom

Arquivos da aula: https://drive.google.com/drive/folders/1FbDqVq4XLvU85VBlVIMJ73p9oOu6u2-J?usp=drive_link

In [10]:
# score de credito do cliente
# Boa - Good
# Ok - Standard
# Ruim - Poor
# Passo a passo
# Passo 0: Entender o desafio e a empresa
# Passo 1: Imporetar a base de dados

# coluna profissão is object
# coluna mix credit is object
# coluna comportamento_pagamento is object



from IPython.display import display
import pandas as pd

# Ajuste o encoding se necessário: encoding="latin-1" ou "utf-8-sig"
tabela = pd.read_csv("clientes.csv")

display(tabela.head())
tabela.info()



,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.46538,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.46538,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.46538,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.46538,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.46538,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

In [11]:
# Importa o LabelEncoder, que transforma valores categóricos (texto) em números inteiros.
# Ex.: ['Engenheiro', 'Médico', 'Professor'] -> [0, 1, 2]
from sklearn.preprocessing import LabelEncoder

# Lista das colunas categóricas que queremos codificar.
cols = ['profissao', 'mix_credito', 'comportamento_pagamento']

# Para cada coluna da lista...
for col in cols:
    # Validação: garante que a coluna existe no DataFrame 'tabela'.
    if col not in tabela.columns:
        raise KeyError(f'Coluna não encontrada: {col}')
    # Converte a coluna para string e preenche valores ausentes com um rótulo padrão.
    # Isso evita erro do LabelEncoder com NaN e padroniza o tipo.
    tabela[col] = tabela[col].astype('string').fillna('desconhecido')

# Cria um encoder para cada coluna (boa prática: 1 encoder por feature).
enc_prof = LabelEncoder()
enc_mix  = LabelEncoder()
enc_comp = LabelEncoder()

# Aplica o fit_transform em cada coluna:
# - fit() aprende os rótulos existentes na coluna
# - transform() converte cada categoria no número correspondente
tabela['profissao'] = enc_prof.fit_transform(tabela['profissao'])
tabela['mix_credito'] = enc_mix.fit_transform(tabela['mix_credito'])
tabela['comportamento_pagamento'] = enc_comp.fit_transform(tabela['comportamento_pagamento'])

# Mostra as primeiras linhas das colunas já codificadas para conferência.
from IPython.display import display
display(tabela[['profissao','mix_credito','comportamento_pagamento']].head())

,profissao,mix_credito,comportamento_pagamento
0,2,0,1
1,2,0,3
2,2,0,5
3,2,0,4
4,2,0,2


In [12]:
# Separar as informações da base de dados para treinar a IA (modelo de machine learning)

# y = variável alvo (o que queremos prever): a coluna 'score_credito'
y = tabela['score_credito']

# x = features (as informações usadas para prever o score): 
# removemos a coluna alvo e também 'id_cliente' (identificador não é uma característica útil e pode causar viés)
x = tabela.drop(columns=['score_credito', 'id_cliente'])

# Separar os dados em treino e teste.
# - train_test_split faz a divisão aleatória do dataset.
# - test_size=0.3 significa 30% para teste e 70% para treino.
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(
    x, y, 
    test_size=0.3
)

In [13]:
# Passo 4: Escolher o melhor modelo de inteligencia artificial

# Arvore de decisão -> Random Forest
# Vizinhos proximos -> KNN



# Passo 1 - Importar o modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Passo 2 - Criar o modelo
modelo_arvore = RandomForestClassifier()
modelo_knn = KNeighborsClassifier()

# Passo 3 - Treinar o modelo
modelo_arvore.fit(x_treino, y_treino)
modelo_knn.fit(x_treino, y_treino)

previsao_arvoreDecisao = modelo_arvore.predict(x_teste)
previsao_knn = modelo_knn.predict(x_teste)

# Acuracia
display(accuracy_score(y_teste, previsao_arvoreDecisao))
display(accuracy_score(y_teste, previsao_knn))



0.8263

0.7352

In [ ]:
from IPython.display import display
import pandas as pd

# 1) Carregar amostra nova (dados que queremos classificar/obter previsões)
# Se ocorrer erro de encoding, teste: encoding="latin-1" ou "utf-8-sig"
tabela_novos_clientes = pd.read_csv("novos_clientes.csv")
display(tabela_novos_clientes.head())  # conferência rápida das primeiras linhas

# 2) Preparar as colunas categóricas para o mesmo pré-processamento do treino
# Essas são as features categóricas que receberam LabelEncoder no treino
cols = ['profissao', 'mix_credito', 'comportamento_pagamento']

for col in cols:
    # Validação: garante que a coluna existe nos dados novos
    if col not in tabela_novos_clientes.columns:
        raise KeyError(f'Coluna não encontrada: {col}')
    # Padroniza o tipo e trata valores ausentes para evitar erro na transformação
    tabela_novos_clientes[col] = tabela_novos_clientes[col].astype('string').fillna('desconhecido')

# 3) Aplicar os MESMOS encoders usados no treino (sem refazer fit!)
# Importante: usamos .transform e não .fit_transform para manter o mesmo mapeamento de rótulos
tabela_novos_clientes['profissao'] = enc_prof.transform(tabela_novos_clientes['profissao'])
tabela_novos_clientes['mix_credito'] = enc_mix.transform(tabela_novos_clientes['mix_credito'])
tabela_novos_clientes['comportamento_pagamento'] = enc_comp.transform(tabela_novos_clientes['comportamento_pagamento'])

# 4) Montar X_novos com as MESMAS colunas do treino, na MESMA ordem
# Removemos a coluna alvo (se existir) e identificadores que não entram no modelo
X_novos = tabela_novos_clientes.drop(columns=['score_credito', 'id_cliente'], errors='ignore')

# Reindex garante que X_novos tenha exatamente as colunas de x_treino
# - Cria colunas faltantes com 0
# - Remove colunas extras
# - Reordena para a mesma ordem do treino
X_novos = X_novos.reindex(columns=x_treino.columns, fill_value=0)

# 5) Gerar previsões com o modelo já treinado nesta sessão
previsoes = modelo_arvore.predict(X_novos)
assert len(previsoes) == len(tabela_novos_clientes)  # segurança: tamanhos devem bater

# Anexamos a previsão ao DataFrame original, preservando todas as colunas
tabela_novos_clientes['score_credito_previsto'] = previsoes

# Exibir todas as colunas (incluindo a nova)
display(tabela_novos_clientes.head())

# (Opcional) Mostrar a coluna de previsão por último na visualização
# A expressão abaixo reordena as colunas apenas para exibição,
# colocando 'score_credito_previsto' no final.
cols_ordenadas = [c for c in tabela_novos_clientes.columns if c != 'score_credito_previsto'] + ['score_credito_previsto']
display(tabela_novos_clientes[cols_ordenadas].head())

# (Opcional) Salvar o resultado completo com a previsão
# tabela_novos_clientes.to_csv('novos_clientes_previstos.csv', index=False)

,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,taxa_uso_credito,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,1,31.0,empresario,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,29.934186,218.0,44.50951,baixo_gasto_pagamento_baixo,312.487689,1,1,0,0,0
1,4,32.0,advogado,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,28.819407,12.0,0.00000,baixo_gasto_pagamento_medio,300.994163,0,0,0,0,1
2,2,48.0,empresario,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,34.235853,215.0,0.00000,baixo_gasto_pagamento_alto,345.081577,0,1,0,1,0


,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil,score_credito_previsto
0,1,31.0,1,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,218.0,44.50951,1,312.487689,1,1,0,0,0,Poor
1,4,32.0,0,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,12.0,0.00000,2,300.994163,0,0,0,0,1,Good
2,2,48.0,1,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,215.0,0.00000,0,345.081577,0,1,0,1,0,Standard


,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil,score_credito_previsto
0,1,31.0,1,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,218.0,44.50951,1,312.487689,1,1,0,0,0,Poor
1,4,32.0,0,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,12.0,0.00000,2,300.994163,0,0,0,0,1,Good
2,2,48.0,1,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,215.0,0.00000,0,345.081577,0,1,0,1,0,Standard
